In [1]:
from pathlib import Path
import json 
import yaml

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [3]:
datasets = [
    "kuhar",
    "motionsense",
    "wisdm",
    "uci",
    "realworld",
    "extrasensory"
]


labels_activity = {
    0: "sit",
    1: "stand",
    2: "walk",
    3: "stair up",
    4: "stair down",
    5: "run",
    6: "stair up and down",
}

# Filters

def filter_same_train_test(df):
    df["anded"] = False
    for d in datasets:
        df["anded"] |= df[f"{d} - train"] & df[f"{d} - test"]
    df = df.loc[df["anded"] == False]
    return df.drop(columns=["anded"])


def drop_es_on_train(df):
    return df.loc[df["extrasensory - train"] == False]

def only_fft(df):
    return df.loc[df["transforms"] == "with_fft_all.0"]

def only_time(df):
    return df.loc[df["transforms"] == "no_transform.0"]

def only_all_axis(df):
    return df.loc[
        (df["use accel-x"]) & (df["use accel-y"]) & (df["use accel-z"]) &
        (df["use gyro-x"]) & (df["use gyro-y"]) & (df["use gyro-z"])
    ]

In [4]:
results_file = Path("results.csv")
results = pd.read_csv(results_file)

In [5]:
results

,execution_id,classification_time,load_time,reduce_time,transform_time,reduce_on,classifier,reducer,transforms,number runs,reducer_datasets,train_datasets,test_datasets,sensors used,umap components,use accel-x,use accel-y,use accel-z,use gyro-x,use gyro-y,use gyro-z,kuhar - reduce,motionsense - reduce,wisdm - reduce,uci - reduce,realworld - reduce,extrasensory - reduce,kuhar - train,motionsense - train,wisdm - train,uci - train,realworld - train,extrasensory - train,kuhar - test,motionsense - test,wisdm - test,uci - test,realworld - test,extrasensory - test,accuracy (mean),accuracy (std),f1-score macro (mean),f1-score macro (std),f1-score micro (mean),f1-score micro (std),f1-score weighted (mean),f1-score weighted (std)
0,24088,10.166383,18.819073,175.687217,0.000037,all,randomforest-100,umap-8-all,with_fft_all.0,5,"wisdm, realworld",realworld,realworld,"accel-x, accel-y, accel-z",8,True,True,True,False,False,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,0.847550,0.001064,0.849012,0.001054,0.847550,0.001064,0.849012,0.001054
1,26052,12.256536,18.711062,174.962821,0.000027,all,randomforest-100,umap-9-all,no_transform.0,5,"kuhar, motionsense","kuhar, motionsense",realworld,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",9,True,True,True,True,True,True,True,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,0.377390,0.002085,0.366682,0.001794,0.377390,0.002085,0.366682,0.001794
2,9720,18.254131,18.766251,129.908457,0.000048,all,randomforest-100,umap-4-all,with_fft_all.0,5,kuhar,"kuhar, motionsense",kuhar,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",4,True,True,True,True,True,True,True,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,0.628155,0.002968,0.614613,0.003303,0.628155,0.002968,0.614613,0.003303
3,25329,8.813036,18.834796,134.984876,0.000035,all,randomforest-100,umap-9-all,no_transform.0,5,realworld,kuhar,realworld,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",9,True,True,True,True,True,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,0.359980,0.002004,0.328627,0.002734,0.359980,0.002004,0.328627,0.002734
4,418,11.276187,18.827668,169.724341,0.000047,all,randomforest-100,umap-2-all,no_transform.0,5,"wisdm, realworld",wisdm,motionsense,"accel-x, accel-y, accel-z",2,True,True,True,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,0.456364,0.000895,0.389048,0.000736,0.456364,0.000895,0.453889,0.000859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10807,31262,23.699687,13.474812,258.166265,0.000022,all,randomforest-100,umap-10-all,with_fft_all.0,5,kuhar,realworld,realworld,"gyro-x, gyro-y, gyro-z",10,False,False,False,True,True,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,0.760261,0.003325,0.759243,0.003352,0.760261,0.003325,0.759243,0.003352
10808,3823,5.132346,42.337780,74.739442,0.000023,all,randomforest-100,umap-3-all,no_transform.0,5,"kuhar, motionsense",motionsense,motionsense,"accel-x, accel-y, accel-z",3,True,True,True,False,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,0.821877,0.000801,0.822824,0.000874,0.821877,0.000801,0.822824,0.000874
10809,5311,23.294950,24.560054,207.458685,0.000027,all,randomforest-100,umap-3-all,no_transform.0,5,kuhar,"wisdm, realworld",uci,"accel-x, accel-y, accel-z",3,True,True,True,False,False,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,0.318310,0.002304,0.227208,0.002054,0.318310,0.002304,0.318091,0.002876
10810,25414,9.208790,15.060317,16.65

In [6]:
x = filter_same_train_test(results)
x = x.sort_values(by="accuracy (mean)").reset_index(drop=True)
x

,execution_id,classification_time,load_time,reduce_time,transform_time,reduce_on,classifier,reducer,transforms,number runs,reducer_datasets,train_datasets,test_datasets,sensors used,umap components,use accel-x,use accel-y,use accel-z,use gyro-x,use gyro-y,use gyro-z,kuhar - reduce,motionsense - reduce,wisdm - reduce,uci - reduce,realworld - reduce,extrasensory - reduce,kuhar - train,motionsense - train,wisdm - train,uci - train,realworld - train,extrasensory - train,kuhar - test,motionsense - test,wisdm - test,uci - test,realworld - test,extrasensory - test,accuracy (mean),accuracy (std),f1-score macro (mean),f1-score macro (std),f1-score micro (mean),f1-score micro (std),f1-score weighted (mean),f1-score weighted (std)
0,18171,6.627275,16.368604,245.963017,0.000026,all,randomforest-100,umap-7-all,no_transform.0,5,"motionsense, wisdm",kuhar,extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",7,True,True,True,True,True,True,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,0.004093,0.000418,0.011179,0.001004,0.004093,0.000418,0.006599,0.000782
1,21772,7.341814,14.137492,219.889114,0.000026,all,randomforest-100,umap-8-all,no_transform.0,5,"motionsense, wisdm",kuhar,extrasensory,"accel-x, accel-y, accel-z",8,True,True,True,False,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,0.004789,0.000449,0.012609,0.002036,0.004789,0.000449,0.007844,0.000949
2,22851,19.629364,15.417859,289.704941,0.000030,all,randomforest-100,umap-8-all,no_transform.0,5,"motionsense, wisdm","kuhar, realworld",extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",8,True,True,True,True,True,True,False,True,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,0.005436,0.000444,0.016391,0.000960,0.005436,0.000444,0.009351,0.000805
3,15090,14.047142,10.765341,324.813820,0.000027,all,randomforest-100,umap-6-all,no_transform.0,5,kuhar,realworld,extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",6,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,0.006659,0.001540,0.010786,0.001905,0.006659,0.001540,0.009739,0.002011
4,23578,5.353522,13.468739,222.959190,0.000018,all,randomforest-100,umap-8-all,with_fft_all.0,5,"wisdm, realworld",kuhar,extrasensory,"accel-x, accel-y, accel-z",8,True,True,True,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,0.006947,0.002296,0.006889,0.001228,0.006947,0.002296,0.012625,0.004578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7935,12933,6.089614,9.540578,88.481451,0.000025,all,randomforest-100,umap-5-all,with_fft_all.0,5,motionsense,motionsense,extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",5,True,True,True,True,True,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,0.781413,0.020955,0.148486,0.002081,0.781413,0.020955,0.784410,0.010901
7936,5031,50.565266,11.851952,129.545138,0.000026,all,randomforest-100,umap-3-all,no_transform.0,5,"motionsense, wisdm","motionsense, wisdm",extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",3,True,True,True,True,True,True,False,True,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,0.782221,0.000743,0.134429,0.000222,0.782221,0.000743,0.793406,0.000327
7937,9516,17.980985,10.682302,105.808013,0.000027,all,randomforest-100,umap-4-all,with_fft_all.0,5,wisdm,wisdm,extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",4,True,True,True,True,True,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,0.786362,0.011910

In [7]:
x["datasets"] = x[["reducer_datasets", "train_datasets", "test_datasets"]].apply("/".join, axis=1)
x

,execution_id,classification_time,load_time,reduce_time,transform_time,reduce_on,classifier,reducer,transforms,number runs,reducer_datasets,train_datasets,test_datasets,sensors used,umap components,use accel-x,use accel-y,use accel-z,use gyro-x,use gyro-y,use gyro-z,kuhar - reduce,motionsense - reduce,wisdm - reduce,uci - reduce,realworld - reduce,extrasensory - reduce,kuhar - train,motionsense - train,wisdm - train,uci - train,realworld - train,extrasensory - train,kuhar - test,motionsense - test,wisdm - test,uci - test,realworld - test,extrasensory - test,accuracy (mean),accuracy (std),f1-score macro (mean),f1-score macro (std),f1-score micro (mean),f1-score micro (std),f1-score weighted (mean),f1-score weighted (std),datasets
0,18171,6.627275,16.368604,245.963017,0.000026,all,randomforest-100,umap-7-all,no_transform.0,5,"motionsense, wisdm",kuhar,extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",7,True,True,True,True,True,True,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,0.004093,0.000418,0.011179,0.001004,0.004093,0.000418,0.006599,0.000782,"motionsense, wisdm/kuhar/extrasensory"
1,21772,7.341814,14.137492,219.889114,0.000026,all,randomforest-100,umap-8-all,no_transform.0,5,"motionsense, wisdm",kuhar,extrasensory,"accel-x, accel-y, accel-z",8,True,True,True,False,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,0.004789,0.000449,0.012609,0.002036,0.004789,0.000449,0.007844,0.000949,"motionsense, wisdm/kuhar/extrasensory"
2,22851,19.629364,15.417859,289.704941,0.000030,all,randomforest-100,umap-8-all,no_transform.0,5,"motionsense, wisdm","kuhar, realworld",extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",8,True,True,True,True,True,True,False,True,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,0.005436,0.000444,0.016391,0.000960,0.005436,0.000444,0.009351,0.000805,"motionsense, wisdm/kuhar, realworld/extrasensory"
3,15090,14.047142,10.765341,324.813820,0.000027,all,randomforest-100,umap-6-all,no_transform.0,5,kuhar,realworld,extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",6,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,0.006659,0.001540,0.010786,0.001905,0.006659,0.001540,0.009739,0.002011,kuhar/realworld/extrasensory
4,23578,5.353522,13.468739,222.959190,0.000018,all,randomforest-100,umap-8-all,with_fft_all.0,5,"wisdm, realworld",kuhar,extrasensory,"accel-x, accel-y, accel-z",8,True,True,True,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,0.006947,0.002296,0.006889,0.001228,0.006947,0.002296,0.012625,0.004578,"wisdm, realworld/kuhar/extrasensory"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7935,12933,6.089614,9.540578,88.481451,0.000025,all,randomforest-100,umap-5-all,with_fft_all.0,5,motionsense,motionsense,extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",5,True,True,True,True,True,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,0.781413,0.020955,0.148486,0.002081,0.781413,0.020955,0.784410,0.010901,motionsense/motionsense/extrasensory
7936,5031,50.565266,11.851952,129.545138,0.000026,all,randomforest-100,umap-3-all,no_transform.0,5,"motionsense, wisdm","motionsense, wisdm",extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",3,True,True,True,True,True,True,False,True,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,0.782221,0.000743,0.134429,0.000222,0.782221,0.000743,0.793406,0.000327,"motionsense, wisdm/motionsense, wisdm/extrasen..."
7937,9516,17.980985,10.682302,

In [8]:
good_treshold = 0.70
y = x.loc[x["accuracy (mean)"] >= good_treshold]
y.datasets.value_counts()

wisdm/wisdm/extrasensory                                  10
motionsense/motionsense/extrasensory                       5
motionsense, realworld/motionsense/extrasensory            4
motionsense, wisdm/motionsense, wisdm/extrasensory         3
motionsense/kuhar, motionsense/extrasensory                2
motionsense, realworld/kuhar, motionsense/extrasensory     2
wisdm, realworld/wisdm, realworld/extrasensory             2
wisdm, realworld/wisdm/extrasensory                        1
realworld/wisdm/extrasensory                               1
Name: datasets, dtype: int64

In [9]:
good_treshold = 0.70
y = x.loc[x["accuracy (mean)"] >= good_treshold]
y["sensors used"].value_counts()

accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z    16
accel-x, accel-y, accel-z                            14
Name: sensors used, dtype: int64

In [10]:
good_treshold = 0.70
y = x.loc[x["accuracy (mean)"] >= good_treshold]
y.groupby("reduce_on")["accuracy (mean)"].mean()

reduce_on
all    0.750406
Name: accuracy (mean), dtype: float64

In [11]:
good_treshold = 0.70
y = x.loc[x["accuracy (mean)"] >= good_treshold]
y["transforms"].value_counts()

no_transform.0    18
with_fft_all.0    12
Name: transforms, dtype: int64

In [12]:
good_treshold = 0.70
y = x.loc[x["accuracy (mean)"] >= good_treshold]
y = y.loc[(y["datasets"] == "wisdm/wisdm/extrasensory") & (y["transforms"] == "with_fft_all.0")]
y

,execution_id,classification_time,load_time,reduce_time,transform_time,reduce_on,classifier,reducer,transforms,number runs,reducer_datasets,train_datasets,test_datasets,sensors used,umap components,use accel-x,use accel-y,use accel-z,use gyro-x,use gyro-y,use gyro-z,kuhar - reduce,motionsense - reduce,wisdm - reduce,uci - reduce,realworld - reduce,extrasensory - reduce,kuhar - train,motionsense - train,wisdm - train,uci - train,realworld - train,extrasensory - train,kuhar - test,motionsense - test,wisdm - test,uci - test,realworld - test,extrasensory - test,accuracy (mean),accuracy (std),f1-score macro (mean),f1-score macro (std),f1-score micro (mean),f1-score micro (std),f1-score weighted (mean),f1-score weighted (std),datasets
7922,5917,13.894960,15.644570,135.852840,0.000021,all,randomforest-100,umap-3-all,with_fft_all.0,5,wisdm,wisdm,extrasensory,"accel-x, accel-y, accel-z",3,True,True,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,0.743233,0.004167,0.182114,0.000265,0.743233,0.004167,0.775197,0.002104,wisdm/wisdm/extrasensory
7925,13116,19.025968,9.779650,103.872057,0.000031,all,randomforest-100,umap-5-all,with_fft_all.0,5,wisdm,wisdm,extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",5,True,True,True,True,True,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,0.748013,0.010951,0.181337,0.001306,0.748013,0.010951,0.774813,0.006220,wisdm/wisdm/extrasensory
7928,5916,13.196632,15.901647,141.245802,0.000038,all,randomforest-100,umap-3-all,with_fft_all.0,5,wisdm,wisdm,extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",3,True,True,True,True,True,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,0.752250,0.005792,0.182500,0.001012,0.752250,0.005792,0.777006,0.003359,wisdm/wisdm/extrasensory
7937,9516,17.980985,10.682302,105.808013,0.000027,all,randomforest-100,umap-4-all,with_fft_all.0,5,wisdm,wisdm,extrasensory,"accel-x, accel-y, accel-z, gyro-x, gyro-y, gyro-z",4,True,True,True,True,True,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,0.786362,0.011910,0.186306,0.001386,0.786362,0.011910,0.794990,0.006252,wisdm/wisdm/extrasensory
